In [1]:
import numpy as np
import pandas as pd

In [2]:
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

# Download data
print(f'Downloading OIH_adjusted.txt...')
urlretrieve('http://api.kibot.com/?action=history&symbol=OIH&interval=1&unadjusted=0&bp=1&user=guest', 'OIH_adjusted.txt')

# Read data and assign names to the columns
df = pd.read_csv('OIH_adjusted.txt')
df.columns = ['date','time','open','high','low','close','volume']

# Combine date and time in the date column
df['date'] = df['date'] + ' ' + df['time']
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M')
df = df[['date','open','high','low','close','volume']]

# Sort by date and assign the date as index
df = df.sort_values('date').reset_index(drop=True).set_index('date')

# Convert the data to different timeframes & save them for future uses
AGGREGATION = {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}
TIMEFRAMES = ['5T', '15T', '1H', '1D']

for timeframe in TIMEFRAMES:
    print(f'Converting & Saving {timeframe} Data...')
    df = df.resample(timeframe).agg(AGGREGATION).dropna()
    df.to_csv(f'OIH_{timeframe}.csv.gz', compression='gzip')


Converting & Saving 5T Data...
Converting & Saving 15T Data...
Converting & Saving 1H Data...
Converting & Saving 1D Data...


In [3]:
TIMEFRAMES = ['5T', '15T', '1H', '1D']

In [4]:
# Using this method, you can obtain buy and sell signals determined by the selected strategy.
# The resulting signals are represented as a series of numerical values:
#   '1' indicating a buy signal,
#   '0' indicating a hold signal, and
#   '-1' indicating a sell signal
def get_signals(df):

    # Buy Signal
    df['signal'] = np.where((df['low'] < df['low'].shift()) & (df['close'] > df['high'].shift()) & (df['open'] < df['close'].shift()), 1, 0)

    # Sell Signal
    df['signal'] = np.where((df['high'] > df['high'].shift()) & (df['close'] < df['low'].shift()) & (df['open'] > df['open'].shift()), -1, df['signal'])

    return df['signal']

# Using this method, you can visualize the results of a simulated long position strategy.
# Note that it assumes the purchase of one share per transaction and does not account for any fees.
def show_stategy_result(timeframe, df):

    waiting_for_close = False
    open_price = 0

    profit = 0.0
    wins = 0
    losses = 0

    for i in range(len(df)):

        signal = df.iloc[i]['signal']

        if signal == 1 and not waiting_for_close:
            waiting_for_close = True
            open_price = df.iloc[i]['close']

        elif signal == -1 and waiting_for_close:
            waiting_for_close = False
            close_price = df.iloc[i]['close']

            profit += close_price - open_price
            wins = wins + (1 if (close_price - open_price) > 0 else 0)
            losses = losses + (1 if (close_price - open_price) < 0 else 0)

    print(f' Result for timeframe {timeframe} '.center(60, '*'))
    print(f'* Profit/Loss: {profit:.2f}')
    print(f"* Wins: {wins} - Losses: {losses}")
    print(f"* Win Rate: {100 * (wins/(wins + losses)):6.2f}%")


In [5]:
# Iterate over each timeframe, apply the strategy and show the result
for timeframe in TIMEFRAMES:

    # Read the data
    df = pd.read_csv(f'OIH_{timeframe}.csv.gz', compression='gzip')

    # Add the signals to each row
    df['signal'] = get_signals(df)

    # Get the result of the strategy
    show_stategy_result(timeframe, df)

***************** Result for timeframe 5T ******************
* Profit/Loss: -398.47
* Wins: 1676 - Losses: 1861
* Win Rate:  47.38%
***************** Result for timeframe 15T *****************
* Profit/Loss: -237.29
* Wins: 617 - Losses: 787
* Win Rate:  43.95%
***************** Result for timeframe 1H ******************
* Profit/Loss: -4.06
* Wins: 292 - Losses: 290
* Win Rate:  50.17%
***************** Result for timeframe 1D ******************
* Profit/Loss: 130.31
* Wins: 24 - Losses: 19
* Win Rate:  55.81%
